# COVID 19 classification


RM - Version 3, February 2021, working on fastai v1, using HMH hospitales

- 17 Feb, Check weather 2 or 3 classes obtain better results
- 17 Feb, using efficient net. 
- 15 Feb Training using bimcv+ and hmhospitales datset and rsna pneumonia challenge.
- 11 Feb: triex two class Negative vs Positive not very good results (low especificity, high sensitivity).
 



In [37]:
import os
os.environ['CUDA_DEVICE_ORDER']='PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES']='1'

In [36]:
import notebook_util 
notebook_util.pick_gpu_lowest_memory()


0

In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
# from fastai.vision.all import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import shutil
import fastai; fastai.__version__
from fastai.vision import *
from fastai.metrics import error_rate
import gc

from efficientnet_pytorch import EfficientNet


In [4]:
bs = 4
# bs = 16   # uncomment this line if you run out of memory even after clicking Kernel->Restart
size_1 = 256
size_2 = 512
classes_2= True


## data

14 classes taken from

https://www.kaggle.com/c/vinbigdata-chest-xray-abnormalities-detection/data

0 - Aortic enlargement
1 - Atelectasis
2 - Calcification
3 - Cardiomegaly
4 - Consolidation
5 - ILD
6 - Infiltration
7 - Lung Opacity
8 - Nodule/Mass
9 - Other lesion
10 - Pleural effusion
11 - Pleural thickening
12 - Pneumothorax
13 - Pulmonary fibrosis

In [5]:
path ='/home/robert/data/covid/'; path
pneumo_labels = "pneumo|infil|consol|fibros|COVID|opacit|glass|pleural|interstitial"
# pneumo_labels = "pneumo|infil|consol|fibros|COVID|opacit|glass|pleural|interstitial"
normal_lables = "normals"


### Load Normal from RSNA and PadChest

Loading normals and pneumos from RSNA and padchest

In [6]:
dfTrain = pd.read_csv(path+'train_RSNA_processed.csv', header=None)
dfTrain['valid'] = 0
dfTrain[0] = 'train_RSNA/processed/' + dfTrain[0].astype(str)
print("Pneumonia:",sum(dfTrain[1]=='pneumonia'))
print("Normals : ",sum(dfTrain[1]=='normal'))
print('Normal train '+str(len (dfTrain)))

# # # adding padchest
dfTrain_padchest = pd.read_csv(path+'padchest/train_split_np.csv', header=None)
dfTrain_padchest['valid'] = 0
dfTrain_padchest[0] = 'padchest/trainNP/processed/' + dfTrain_padchest[0].astype(str)

dfTrain = dfTrain.append(dfTrain_padchest)

dfTrain.rename(columns={1: "Class"},  inplace=True)
dfTrain.rename(columns={0: "File"},  inplace=True)



print("Pneumonia:",sum(dfTrain["Class"]=='pneumonia'))
print("Normals : ",sum(dfTrain["Class"]=='normal'))
print('Total train '+str(len (dfTrain)))
dfTrain.head()


Pneumonia: 5346
Normals :  7962
Normal train 13308
Pneumonia: 8300
Normals :  13443
Total train 21743


,File,Class,valid
0,train_RSNA/processed/003d8fa0-6bf1-40ed-b54c-a...,normal,0
1,train_RSNA/processed/009eb222-eabc-4150-8121-d...,normal,0
2,train_RSNA/processed/01027bc3-dc40-4165-a6c3-d...,normal,0
3,train_RSNA/processed/017c7b5b-618e-4bc9-943c-0...,normal,0
4,train_RSNA/processed/019ca122-9cdf-4704-b7a9-4...,normal,0


### Load Normal  BIMCV COVID Neg.

Adding normals and multiple patologies from bimcv negative


In [7]:
dfTrain_bim = pd.read_csv(path+'bimcv_covidN_db/bimcv_neg_train_proc.csv', header=0)
dfTrain_bim['valid'] = 0
dfTrain_bim['File'] = 'bimcv_covidN_db/processed/' + dfTrain_bim['File'].astype(str)+'_8b.png'
dfTrain_bim['Class']  = "normal"




/home/robert/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (20) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Filling the labels: removing Nans


In [8]:
# dfTrain_bim = dfTrain_bim.replace(np.nan, '', regex=True)
dfTrain_bim.Labels.fillna("normal", inplace=True)
dfTrain_bim.loc[dfTrain_bim['Labels'].str.contains(pneumo_labels),"Class"] = "pneumonia"


In [9]:
print("Pneumonia:",sum(dfTrain_bim["Class"]=='pneumonia'))
print("Normals : ",sum(dfTrain_bim["Class"]=='normal'))


print('Normal train '+str(len (dfTrain_bim)))
dfTrain_bim.head()

Pneumonia: 1913
Normals :  1621
Normal train 3534


,Unnamed: 0,Subject,Session,mod,File,Type,Position_Manual,Exclude,Exclude Reason,Labels,...,Detector Type,Imager Pixel Spacing,Exposure Index,Filter Thickness Maximum,Performed Procedure Step Start Time,Bits Stored,Study Instance UID,Patient Gantry Relationship Code Sequence,valid,Class
0,0,sub-S06039,ses-E10945,mod-rx,bimcv_covidN_db/processed/sub-S06039_ses-E1094...,cr,PA,False,NaN,unchanged,...,NaN,"0.143, 0.143",NaN,NaN,182705.234,12.0,1.2.840.113654.2.70.1.107288151586885502570645...,NaN,0,normal
1,2,sub-S06110,ses-E11058,mod-rx,bimcv_covidN_db/processed/sub-S06110_ses-E1105...,dx,PA,False,NaN,infiltrates,...,SCINTILLATOR,"0.175, 0.175",67.560167,NaN,212816.000,12.0,1.2.840.113654.2.70.1.216859291845841333754195...,NaN,0,pneumonia
2,3,sub-S06113,ses-E11062,mod-rx,bimcv_covidN_db/processed/sub-S06113_ses-E1106...,dx,PA,False,NaN,'\t 'heart insufficiency'\t ' interstitial pat...,...,NaN,"0.138999998569489, 0.138999998569489",NaN,NaN,NaN,16.0,1.2.840.113654.2.70.1.177717506051411834280838...,NaN,0,pneumonia
3,4,sub-S06116,ses-E11066,mod-rx,bimcv_covidN_db/processed/sub-S06116_ses-E1106...,cr,PA,False,NaN,suboptimal study'\t 'fibrotic band'\t 'COPD si...,...,NaN,"0.1, 0.1",NaN,NaN,NaN,12.0,1.2.840.113654.2.70.1.287550085791502471302689...,NaN,0,pneumonia
4,5,sub-S06145,ses-E11104,mod-rx,bimcv_covidN_db/processed/sub-S06145_ses-E1110...,dx,PA,False,NaN,interstitial pattern,...,SCINTILLATOR,"0.194553, 0.194553",NaN,NaN,115109.000,14.0,1.2.840.113654.2.70.1.227917490972241029816891...,"{'00080100': {'Value': F-10470, 'vr': 'SH'}, '...",0,pneumonia


In [10]:
dfTrain_bim = dfTrain_bim[['File', 'Class', 'valid']]
dfTrain = dfTrain.append(dfTrain_bim)

print("TOTAL Train Pneumonia:",sum(dfTrain["Class"]=='pneumonia'))
print("TOTAL Train Normals : ",sum(dfTrain["Class"]=='normal'))


print('TOTAL Normal train '+str(len (dfTrain)))
dfTrain.head()


TOTAL Train Pneumonia: 10213
TOTAL Train Normals :  15064
TOTAL Normal train 25277


,File,Class,valid
0,train_RSNA/processed/003d8fa0-6bf1-40ed-b54c-a...,normal,0
1,train_RSNA/processed/009eb222-eabc-4150-8121-d...,normal,0
2,train_RSNA/processed/01027bc3-dc40-4165-a6c3-d...,normal,0
3,train_RSNA/processed/017c7b5b-618e-4bc9-943c-0...,normal,0
4,train_RSNA/processed/019ca122-9cdf-4704-b7a9-4...,normal,0


### Load COVID for training from hmhospitales

In [11]:
dfTrainCovid = pd.read_csv(path+'hmhospitales/train_hmh_processed.csv', header=None)
dfTrainCovid['valid'] = 0
dfTrainCovid[0] = 'hmhospitales/train/processed/' + dfTrainCovid[0].astype(str)
print("HMH: ", len (dfTrainCovid))

dfTrainv3 = pd.read_csv(path+'train_split_v3.csv', header=None)
dfTrainv3['valid'] = 0
dfTrainv3[0] = 'train/processed/' + dfTrainv3[0].astype(str)
dfTrainv3 = dfTrainv3.loc[dfTrainv3[1] == 'COVID-19']
# # augment de training covid data 2^3
# for i in range(2):
#     dfTrainv3 = dfTrainv3.append(dfTrainv3)
print("Splitv3: ", len (dfTrainv3))
    
dfTrainCovid = dfTrainCovid.append(dfTrainv3)


# # salutms_xnat (217 cases) . Using salut cases!?
# df_salut = pd.read_csv(path+'salutms_xnat/salutms_xnat2.csv',header=None)
# df_salut[0] = 'salutms_xnat/img/processed/' + df_salut[0].astype(str)
# df_salut.loc[df_salut[1] == ('Cardiopatia') , 1] = 'normal'
# df_salut.loc[df_salut[1] == ('Fractura') , 1] = 'normal'
# df_salut = df_salut.drop(2, 1)
# df_train_salut, df_test_salut = train_test_split(df_salut, test_size=0.5)
# # augment de training covid data 2^3
# for i in range(3):
#     df_train_salut = df_train_salut.append(df_train_salut)
# print("Salutms: ", len (df_train_salut))
# dfTrainCovid = dfTrainCovid.append(df_train_salut)

dfTrainCovid.rename(columns={1: "Class"},  inplace=True)
dfTrainCovid.rename(columns={0: "File"},  inplace=True)

HMH:  2030
Splitv3:  271


### Load COVID  BIMCV COVID Pos.

In [12]:
dfTrain_bimp = pd.read_csv(path+'bimcv_covidP_db/bimcv_pos_train_proc.csv', header=0)
dfTrain_bimp['valid'] = 0
dfTrain_bimp['File'] = 'bimcv_covidP_db/processed/' + dfTrain_bimp['File'].astype(str)+'_8b.png'
dfTrain_bimp['Class']  = "COVID-19"
# dfTrain_bimp = dfTrain_bimp.replace(np.nan, '', regex=True)


/home/robert/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (20,24,43,136,178,257) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [13]:
# multi-label? 
# df_salut.loc[df_salut[1] == ('Fractura') , 1] = 'normal'
# dfTrain_bim = dfTrain_bim.replace(np.nan, '', regex=True)
# dfTrain_bim.loc[dfTrain_bim['Labels'].str.contains(pneumo_labels),"Class"] = "pneumonia"


In [14]:
dfTrain_bimp = dfTrain_bimp[['File', 'Class', 'valid']]
print(sum(dfTrain_bimp["Class"]=='COVID-19'))
print(len(dfTrain_bimp))
dfTrain_bimp.head()

8102
8102


,File,Class,valid
0,bimcv_covidP_db/processed/sub-S10212_ses-E1935...,COVID-19,0
1,bimcv_covidP_db/processed/sub-S12131_ses-E2391...,COVID-19,0
2,bimcv_covidP_db/processed/sub-S12131_ses-E2436...,COVID-19,0
3,bimcv_covidP_db/processed/sub-S03059_ses-E2280...,COVID-19,0
4,bimcv_covidP_db/processed/sub-S10901_ses-E1897...,COVID-19,0


In [15]:
dfTrainCovid=dfTrainCovid.append(dfTrain_bimp)

In [16]:
print("Total COVID ", len (dfTrainCovid))

dfTrainCovid.head()

Total COVID  10403


,File,Class,valid
0,hmhospitales/train/processed/1.2.124.113532.12...,COVID-19,0
1,hmhospitales/train/processed/1.2.124.113532.12...,COVID-19,0
2,hmhospitales/train/processed/1.2.124.113532.14...,COVID-19,0
3,hmhospitales/train/processed/1.2.124.113532.14...,COVID-19,0
4,hmhospitales/train/processed/1.2.124.113532.15...,COVID-19,0


## Join Normals, pneumonia and COVIDs for training


In [17]:
dfTrain = dfTrain.append(dfTrainCovid)
print('Total Train',len (dfTrain))
dfTrain.head()

Total Train 35680


,File,Class,valid
0,train_RSNA/processed/003d8fa0-6bf1-40ed-b54c-a...,normal,0
1,train_RSNA/processed/009eb222-eabc-4150-8121-d...,normal,0
2,train_RSNA/processed/01027bc3-dc40-4165-a6c3-d...,normal,0
3,train_RSNA/processed/017c7b5b-618e-4bc9-943c-0...,normal,0
4,train_RSNA/processed/019ca122-9cdf-4704-b7a9-4...,normal,0


### Validation data from RSNA & HMH

In [18]:
dfTest = pd.read_csv(path+'bimcv_covidN_db/bimcv_neg_test_proc.csv', header=0)
dfTest['valid'] = 1
dfTest['File'] = 'bimcv_covidN_db/processed/' + dfTest['File'].astype(str)+'_8b.png'
dfTest['Class']  = "normal"
dfTest = dfTest.replace(np.nan, '', regex=True)
dfTest.loc[dfTest['Labels'].str.contains(pneumo_labels),"Class"] = "pneumonia"

dfTest = dfTest[['File', 'Class', 'valid']]
dfTest.head()




print("Test Pneumonia:",sum(dfTest["Class"]=='pneumonia'))
print("Test Normals : ",sum(dfTest["Class"]=='normal'))

print('Test BIMCV N',len (dfTest))
dfTest.head()

Test Pneumonia: 481
Test Normals :  433
Test BIMCV N 914


,File,Class,valid
0,bimcv_covidN_db/processed/sub-S06150_ses-E1266...,normal,1
1,bimcv_covidN_db/processed/sub-S06583_ses-E1226...,pneumonia,1
2,bimcv_covidN_db/processed/sub-S06591_ses-E1176...,normal,1
3,bimcv_covidN_db/processed/sub-S06728_ses-E1198...,normal,1
4,bimcv_covidN_db/processed/sub-S06969_ses-E1238...,pneumonia,1


Load bimcv positive

In [19]:
dfTestP = pd.read_csv(path+'bimcv_covidP_db/bimcv_pos_test_proc.csv', header=0)
dfTestP['valid'] = 1
dfTestP['File'] = 'bimcv_covidP_db/processed/' + dfTestP['File'].astype(str)+'_8b.png'
dfTestP['Class']  = "COVID-19"
dfTestP = dfTestP.replace(np.nan, '', regex=True)
# dfTest.loc[dfTest['Labels'].str.contains(pneumo_labels),"Class"] = "pneumonia"

dfTestP = dfTestP[['File', 'Class', 'valid']]
dfTestP.head()


print('Test BIMCV P',len (dfTestP))
dfTestP.head()

Test BIMCV P 2039


,File,Class,valid
0,bimcv_covidP_db/processed/sub-S12140_ses-E2395...,COVID-19,1
1,bimcv_covidP_db/processed/sub-S10997_ses-E2387...,COVID-19,1
2,bimcv_covidP_db/processed/sub-S10997_ses-E1925...,COVID-19,1
3,bimcv_covidP_db/processed/sub-S10997_ses-E2590...,COVID-19,1
4,bimcv_covidP_db/processed/sub-S09666_ses-E1656...,COVID-19,1


In [20]:
dfTestCovid = pd.read_csv(path+'hmhospitales/test_hmh_processed.csv',header=None)
dfTestCovid['valid'] = 1
dfTestCovid[0] = 'hmhospitales/test/processed/' + dfTestCovid[0].astype(str)
print('Test HMH ',len (dfTestCovid))

dfTestCovid.head()

Test HMH  534


,0,1,valid
0,hmhospitales/test/processed/1.2.124.113532.161...,COVID-19,1
1,hmhospitales/test/processed/1.2.124.113532.172...,COVID-19,1
2,hmhospitales/test/processed/1.2.124.113532.229...,COVID-19,1
3,hmhospitales/test/processed/1.2.840.113619.2.2...,COVID-19,1
4,hmhospitales/test/processed/1.2.840.113619.2.2...,COVID-19,1


In [21]:
dfTestCovid.rename(columns={1: "Class"},  inplace=True)
dfTestCovid.rename(columns={0: "File"},  inplace=True)
dfTestCovid.head()

,File,Class,valid
0,hmhospitales/test/processed/1.2.124.113532.161...,COVID-19,1
1,hmhospitales/test/processed/1.2.124.113532.172...,COVID-19,1
2,hmhospitales/test/processed/1.2.124.113532.229...,COVID-19,1
3,hmhospitales/test/processed/1.2.840.113619.2.2...,COVID-19,1
4,hmhospitales/test/processed/1.2.840.113619.2.2...,COVID-19,1


Adding all tests

In [22]:
dfTest= dfTest.append(dfTestP)
dfTest= dfTest.append(dfTestCovid)

print('Total Test',len (dfTest))

Total Test 3487


In [23]:
df = dfTrain.append(dfTest)
print(len(df))
df.head()

39167


,File,Class,valid
0,train_RSNA/processed/003d8fa0-6bf1-40ed-b54c-a...,normal,0
1,train_RSNA/processed/009eb222-eabc-4150-8121-d...,normal,0
2,train_RSNA/processed/01027bc3-dc40-4165-a6c3-d...,normal,0
3,train_RSNA/processed/017c7b5b-618e-4bc9-943c-0...,normal,0
4,train_RSNA/processed/019ca122-9cdf-4704-b7a9-4...,normal,0


### Converting to 2 classes?

In [24]:
if (classes_2): 
    # making it two class only normal
    df.loc[df['Class']=="pneumonia","Class"] = "normal"
    
print(" COVIDs ",sum(df["Class"]=='COVID-19'))
print(" Normal ",sum(df["Class"]=='normal'))
print(" Pneumonia ",sum(df["Class"]=='pneumonia'))


 COVIDs  12976
 Normal  26191
 Pneumonia  0


## Data loaders

In [25]:
np.random.seed(42)

src = (ImageList.from_df(df=df,path=path,cols='File')
    .split_from_df(col='valid')
    .label_from_df())


In [26]:
# allowing horizontal flips only. 
tfms = get_transforms(do_flip=True,max_rotate=5,flip_vert=False, max_lighting=0.1, max_zoom=1.05, max_warp=0.)


In [ ]:
data = (src.transform(tfms, size=size_1)
        .databunch().normalize(imagenet_stats))
data.batch_size=bs


In [ ]:
data.show_batch(rows=10,figsize=(15,15))

In [ ]:
data.train_ds

In [ ]:
data.valid_ds


### Convert to grayscale

In [ ]:
for itemList in ["train_dl", "valid_dl", "fix_dl", "test_dl"]:
    itemList = getattr(data, itemList)
    if itemList: itemList.x.convert_mode = "L"

In [ ]:
data.train_ds

## learner

In [31]:
'''class fbeta_binary(Callback):
    "Computes the f_beta between preds and targets for binary text classification"

    def __init__(self, beta2 = 1, eps=1e-9,sigmoid = True):      
        self.beta2=beta2**2
        self.eps = eps
        self.sigmoid = sigmoid
    
    def on_epoch_begin(self, **kwargs):
        self.TP = 0
        self.total_y_pred = 0   
        self.total_y_true = 0
    
    def on_batch_end(self, last_output, last_target, **kwargs):
        y_pred = last_output
        y_pred = y_pred.softmax(dim = 1)        
        y_pred = y_pred.argmax(dim=1)
        y_true = last_target.float()
        
        self.TP += ((y_pred==1) * (y_true==1)).float().sum()
        self.total_y_pred += (y_pred==1).float().sum()
        self.total_y_true += (y_true==1).float().sum()
    
    def on_epoch_end(self, **kwargs):
        prec = self.TP/(self.total_y_pred+self.eps)
        rec = self.TP/(self.total_y_true+self.eps)
        res = (prec*rec)/(prec*self.beta2+rec+self.eps)*(1+self.beta2)        
        #self.metric = res.mean()
        self.metric = res     
'''

'class fbeta_binary(Callback):\n    "Computes the f_beta between preds and targets for binary text classification"\n\n    def __init__(self, beta2 = 1, eps=1e-9,sigmoid = True):      \n        self.beta2=beta2**2\n        self.eps = eps\n        self.sigmoid = sigmoid\n    \n    def on_epoch_begin(self, **kwargs):\n        self.TP = 0\n        self.total_y_pred = 0   \n        self.total_y_true = 0\n    \n    def on_batch_end(self, last_output, last_target, **kwargs):\n        y_pred = last_output\n        y_pred = y_pred.softmax(dim = 1)        \n        y_pred = y_pred.argmax(dim=1)\n        y_true = last_target.float()\n        \n        self.TP += ((y_pred==1) * (y_true==1)).float().sum()\n        self.total_y_pred += (y_pred==1).float().sum()\n        self.total_y_true += (y_true==1).float().sum()\n    \n    def on_epoch_end(self, **kwargs):\n        prec = self.TP/(self.total_y_pred+self.eps)\n        rec = self.TP/(self.total_y_true+self.eps)\n        res = (prec*rec)/(prec*self

In [32]:

#arch = models.resnet50
#arch = models.densenet161
#learn = cnn_learner(data, arch, metrics=[accuracy, Precision(), AUROC(), error_rate])
# using efficient net
# rm model = EfficientNet.from_pretrained("efficientnet-b4", advprop=True)
#model = EfficientNet.from_name("efficientnet-b7")
model = EfficientNet.from_pretrained("efficientnet-b8", advprop=True) # weights run to NaN
#model = EfficientNet.from_name('efficientnet-b4')
#model._fc = nn.Linear(1280, data.c)# the last layer... # works for b0,b1
#model._fc = nn.Linear(1536, data.c)# the last layer... B3
model._fc = nn.Linear(1792, data.c)# the last layer... B4
#model._fc = nn.Linear(2048, data.c)# the last layer... B5
#model._fc = nn.Linear(2304, data.c)# the last layer... B6
#model._fc = nn.Linear(2560, dls.c)# the last layer... B7
model._fc = nn.Linear(2816, data.c)# the last layer... B8


# learn = cnn_learner(data, model, metrics=[accuracy, Precision(), error_rate])
learn = Learner(data, model, metrics=[accuracy, Precision(), error_rate])


Loaded pretrained weights for efficientnet-b8


In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot(suggestion=True)

In [ ]:
lr = 1.0E-03

In [ ]:
learn.fit_one_cycle(10, slice(lr))

In [ ]:
learn.recorder.plot_losses()

In [ ]:
learn.save('stage-1-'+str(size_1)+'-enb8')

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
losses,idxs = interp.top_losses()

len(data.valid_ds)==len(losses)==len(idxs)

In [ ]:
data.classes, data.c, len(data.train_ds), len(data.valid_ds)

In [ ]:
# interp.plot_top_losses(10, figsize=(15,11), heatmap= True)

In [ ]:
interp.plot_confusion_matrix(figsize=(12,12), dpi=60)

In [ ]:
interp.most_confused(min_val=1)

In [ ]:
learn.unfreeze()

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot(suggestion=True)

In [ ]:
print(lr)

In [ ]:
# learn.fit_one_cycle(15,slice(1e-5, lr/2))
learn.fit_one_cycle(15,slice(1e-5))

In [ ]:
learn.recorder.plot_losses()

In [ ]:
learn.save('stage-2-'+str(size_1)+'256-enb8')


In [ ]:

interp = ClassificationInterpretation.from_learner(learn)
losses,idxs = interp.top_losses()

len(data.valid_ds)==len(losses)==len(idxs)
interp.plot_confusion_matrix(figsize=(12,12), dpi=60)

In [ ]:
mat = interp.confusion_matrix()
print(mat)

In [ ]:
diag = sum(mat[i][i] for i in range(len(mat)))
sens = np.zeros(len(mat))
spec = np.zeros(len(mat))
for i in range(len(mat)):
    tp = mat[i,i]
    fp = sum(mat[:,i])-tp
    fn = sum(mat[i,:])-tp
    tn = diag - tp
#     print(tp, fp, fn, tn)
    sens[i] = tp/(tp + fn)
    spec[i] = tn/(fp + tn)
print ("Sensitivity", sens, "Specificity", spec)

### increasing image size

In [ ]:
# learn = []
# torch.cuda.empty_cache()
# gc.collect()


In [30]:


data = (src.transform(tfms, size=size_2)
        .databunch().normalize(imagenet_stats))
for itemList in ["train_dl", "valid_dl", "fix_dl", "test_dl"]:
    itemList = getattr(data, itemList)
    if itemList: itemList.x.convert_mode = "L"


In [ ]:
learn.load ('stage-2-'+str(size_1)+'256-enb8')

In [29]:

data.batch_size = bs
data.train_ds[0][0].shape

torch.Size([1, 512, 512])

In [ ]:
learn.data = data

In [ ]:
learn.freeze()

In [ ]:
learn.lr_find()
learn.recorder.plot()

In [33]:
lr=1e-4/2

In [34]:
learn.fit_one_cycle(10, slice(lr))

epoch,train_loss,valid_loss,accuracy,precision,error_rate,time


RuntimeError: CUDA out of memory. Tried to allocate 512.00 MiB (GPU 0; 10.76 GiB total capacity; 9.72 GiB already allocated; 252.38 MiB free; 9.74 GiB reserved in total by PyTorch)

In [ ]:
learn.recorder.plot_losses()

In [ ]:
learn.save('stage-1-'+str(size_2)+'-enb8')

In [ ]:
learn.unfreeze()

In [ ]:
learn.lr_find()
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(15, slice(1e-3))

In [ ]:
learn.recorder.plot_losses()

In [ ]:
learn.save('stage-2-'+str(size_2)+'-rn50')


In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
losses,idxs = interp.top_losses()

len(data.valid_ds)==len(losses)==len(idxs)
interp.plot_confusion_matrix(figsize=(12,12), dpi=60)

In [ ]:
interp.plot_top_losses(10, figsize=(15,11), largest=True, heatmap= True)

In [ ]:
interp.plot_top_losses(10, figsize=(15,11), largest=False, heatmap= True)

In [ ]:
mat = interp.confusion_matrix()
print(mat)

In [ ]:
diag = sum(mat[i][i] for i in range(len(mat)))
sens = np.zeros(len(mat))
spec = np.zeros(len(mat))
for i in range(len(mat)):
    tp = mat[i,i]
    fp = sum(mat[:,i])-tp
    fn = sum(mat[i,:])-tp
    tn = diag - tp
#     print(tp, fp, fn, tn)
    sens[i] = tp/(tp + fn)
    spec[i] = tn/(fp + tn)
print ("Sensitivity", sens, "Specificity", spec)

In [ ]:
learn.export('trainedAllData_resnet_ROI_'+str(size_2)+'_exportFeb21_effB8_2class.pkl')

### Test on new data

In [ ]:
learn = load_learner(path,'trainedAllData_resnet_ROI_'+str(size_2)+'_exportFeb21.pkl')


In [ ]:
# Data from trueta, Tic Salut, Udiat
dfTest = pd.read_csv(path+'covid_local/covid_local_APonly.csv',header=None)
dfTest[0] = 'covid_local/processed/' + dfTest[0].astype(str)
dfTest.loc[dfTest[1] == ('normal') , 1] = 'normal'
print("Local", len(dfTest))


#CSMS (earlier files <20 cases)
dfTest2 = pd.read_csv(path+'CSMS/CSMS.csv',header=None)
dfTest2[0] = 'CSMS/processed/' + dfTest2[0].astype(str)
dfTest2.loc[dfTest2[1] == ('normal') , 1] = 'normal'

dfTest2.head()

dfTest = dfTest.append(dfTest2)

# # padchest to include normals and pneumonia (not used for training/validation/testing)
dfTest3 = pd.read_csv(path+'padchest/test_split_np.csv',header=None)
dfTest3[0] = 'padchest/testNP/processed/' + dfTest3[0].astype(str)
dfTest3.loc[dfTest3[1] == ('normal') , 1] = 'normal'
dfTest3.loc[dfTest3[1] == ('pneumonia') , 1] = 'normal'
dfTest = dfTest.append(dfTest3)

# salutms (120 cases) all COVID-19 in jpg
dfTest4 = pd.read_csv(path+'salutms/salutms_detect.csv',header=None)
dfTest4[0] = 'salutms/processed/' + dfTest4[0].astype(str)
dfTest = dfTest.append(dfTest4)


# # # salutms_xnat (217 cases) . Using salut cases!?
# df_salut = pd.read_csv(path+'salutms_xnat/salutms_xnat2.csv',header=None)
# df_salut[0] = 'salutms_xnat/img/processed/' + df_salut[0].astype(str)
# df_salut.loc[df_salut[1] == ('Cardiopatia') , 1] = 'normal'
# df_salut.loc[df_salut[1] == ('Fractura') , 1] = 'normal'
# df_salut = df_salut.drop(2, 1)

# # dfTest = dfTest.append(df_salut)
# dfTest = df_salut


dfTest.rename(columns={1: "Class"},  inplace=True)
dfTest.rename(columns={0: "File"},  inplace=True)


In [ ]:
print("Test ",len(dfTest))
print("Normal",sum(dfTest["Class"]=="normal"))
print("COVID-19",sum(dfTest["Class"]=="COVID-19"))

dfTest.tail()


In [ ]:
print(imagenet_stats)

In [ ]:
src = (ImageList.from_df(df=dfTest,path=path,convert_mode='L')
    .split_none()
    .label_from_df())

test = (src.transform([], size=512)
        .databunch(no_check=False,bs=1).normalize(imagenet_stats))
test.train_dl

In [ ]:
test.valid_dl = test.train_dl

test.show_batch(rows=13,figsize=(15,15))

for itemList in ["train_dl", "valid_dl", "fix_dl", "test_dl"]:
    itemList = getattr(test, itemList)
    if itemList: itemList.x.convert_mode = "L"


learn.data.valid_dl = test.train_dl
interp = ClassificationInterpretation.from_learner(learn)
losses,idxs = interp.top_losses()

#len(data.valid_ds)==len(losses)==len(idxs)
interp.plot_confusion_matrix(figsize=(12,12), dpi=60)

In [ ]:
mat = interp.confusion_matrix()
print(mat)

In [ ]:
diag = sum(mat[i][i] for i in range(len(mat)))
sens = np.zeros(len(mat))
spec = np.zeros(len(mat))
for i in range(len(mat)):
    tp = mat[i,i]
    fp = sum(mat[:,i])-tp
    fn = sum(mat[i,:])-tp
    tn = diag - tp
#     print(tp, fp, fn, tn)
    sens[i] = tp/(tp + fn)
    spec[i] = tn/(fp + tn)
print ("Sensitivity", sens, "Specificity", spec)

In [ ]:
log_preds, y = learn.get_preds()

predIdxs = np.argmax(log_preds.numpy(), axis=1)
print(exp(log_preds))



In [ ]:
len(data.valid_ds)
predIdxs
log_preds
predIdxs

In [ ]:
interp.plot_top_losses(25, figsize=(15,11), largest=True, heatmap= True)

In [ ]:
interp.plot_top_losses(25, figsize=(15,11), largest=False, heatmap= True)